In [9]:
#初始化
import os
import toml
from openai import AzureOpenAI

# 读取 secrets.toml 文件
secrets = toml.load('secrets.toml')

client = AzureOpenAI(
  api_key = secrets.get('OPENAI_API_KEY'),
  api_version = secrets.get('AZURE_API_VERSION'),
  azure_endpoint = secrets.get('AZURE_API_BASE')
)

# 设置openai API默认参数, name = gpt-35-turbo-1106
def get_completion(prompt): 
    messages = [
        {"role": "system", "content": """Translate this markdown content to Simplified Chinese. Follow the below rules:
         - Must keep original markdown format and position. 
         - Do not change any markdown or html tags.
         - The expected documentation or material shall keep the integrality of contents.
         - The translated content shall be grammatically correct.
         - The translated content shall be fluent and easy to understand.
         - do not add any additional content.
         - do not remove any content."""},
        {"role": "user", "content": prompt}
        ]
    response = client.chat.completions.create(
      model="gpt-4o",
      messages=messages
      )
    return response.choices[0].message.content


In [10]:

filepath="output/A_Computational_Framework_for_Estimating_GPU_Demand_in_Open-Source_Large_Language_Models/storm_gen_article_polished.txt"

# 获取文件夹路径
folder_path = os.path.dirname(filepath)

# 截取文件上一级目录名作为file_name
file_name = os.path.basename(folder_path)

print(folder_path)
print(file_name)

output/A_Computational_Framework_for_Estimating_GPU_Demand_in_Open-Source_Large_Language_Models
A_Computational_Framework_for_Estimating_GPU_Demand_in_Open-Source_Large_Language_Models


In [11]:
from markitdown import MarkItDown
import os

md = MarkItDown()
result = md.convert(filepath)

# 分割结果文本为自然段，每个空行作为段落的分隔符
paragraphs = [p for p in result.text_content.split("\n\n") if p.strip()]

# 逐段调用 get_completion，并将翻译结果存入列表
translated_paragraphs = []
for para in paragraphs:
    translated_paragraph = get_completion(para)
    translated_paragraphs.append(translated_paragraph)

# 合并翻译后的段落为完整的 Markdown 文本
complete_translated_markdown = "\n\n".join(translated_paragraphs)

# 获取当前工作目录
cwd = os.path.dirname(filepath)
print("当前工作目录：", cwd)

# 保存非翻译结果的 Markdown 文件
output_filepath1 = os.path.join(cwd, 'non-translated_result.md')
try:
    with open(output_filepath1, 'w', encoding='utf-8') as file:
        file.write(result.text_content)
    print(f"非翻译结果已保存至 {output_filepath1}")
except Exception as e:
    print("保存非翻译结果时发生错误：", e)

# 保存翻译后的 Markdown 文件
output_filepath2 = os.path.join(cwd, 'translated_result.md')
try:
    with open(output_filepath2, 'w', encoding='utf-8') as file:
        file.write(complete_translated_markdown)
    print(f"翻译结果已保存至 {output_filepath2}")
except Exception as e:
    print("保存翻译结果时发生错误：", e)

当前工作目录： output/A_Computational_Framework_for_Estimating_GPU_Demand_in_Open-Source_Large_Language_Models
非翻译结果已保存至 output/A_Computational_Framework_for_Estimating_GPU_Demand_in_Open-Source_Large_Language_Models/non-translated_result.md
翻译结果已保存至 output/A_Computational_Framework_for_Estimating_GPU_Demand_in_Open-Source_Large_Language_Models/translated_result.md


In [15]:
#combine the translated result with references for EN and CN

import json
import os

# 文件路径
# folder_path = "How_do_you_measure_the_software_development_performance_gains_from_GitHub_Copilot"

# 读取 JSON 文件
with open(os.path.join(folder_path, 'url_to_info.json'), 'r', encoding='utf-8') as file:
    data = json.load(file)

# 提取标题，适应不同结构
# title = data.get('title') or data.get('info', {}).get('title') or 'default_title'
title = file_name

# 提取引用信息，使用可点击Markdown链接
references = []
for i, (url, info) in enumerate(data.get('url_to_info', {}).items(), start=1):
    ref_title = info.get('title', 'Untitled')
    if ref_title == "PDF" and 'meta' in info and 'query' in info['meta']:
        ref_title = info['meta']['query']
    references.append(f"- [{i}] [{ref_title}]({url})")

# 读取现有的 Markdown 文件内容
with open(os.path.join(folder_path, 'non-translated_result.md'), 'r', encoding='utf-8') as file:
    md_content = file.read()

# 将引用信息附加到 Markdown 文件末尾
output_filepath1 = os.path.join(folder_path, f"{title}_non-translated_result_with_references.md")

with open(output_filepath1, 'w', encoding='utf-8') as file:
    file.write(md_content)
    file.write("\n\n# references\n\n")
    for reference in references:
        file.write(f"{reference}\n")

# 读取现有的 Markdown 文件内容
with open(os.path.join(folder_path, 'translated_result.md'), 'r', encoding='utf-8') as file:
    md_content = file.read()

# 将引用信息附加到 Markdown 文件末尾
output_filepath = os.path.join(folder_path, f"{title}_translated_result_with_references.md")

with open(output_filepath, 'w', encoding='utf-8') as file:
    file.write(md_content)
    file.write("\n\n# 引用\n\n")
    for reference in references:
        file.write(f"{reference}\n")

print(f"References saved to {output_filepath}")

References saved to output/A_Computational_Framework_for_Estimating_GPU_Demand_in_Open-Source_Large_Language_Models/A_Computational_Framework_for_Estimating_GPU_Demand_in_Open-Source_Large_Language_Models_translated_result_with_references.md


In [19]:
import os
from IPython.display import Markdown, display

# 定义 Markdown 文件的路径，这里以 'translated_result.md' 为例
folder_path = os.path.dirname(filepath)
md_file = os.path.join(folder_path, 'translated_result.md')

# 读取 Markdown 文件内容
with open(md_file, 'r', encoding='utf-8') as file:
    md_content = file.read()

# 渲染并显示 Markdown 内容
display(Markdown(md_content))

# 摘要

**一种用于估算开源大型语言模型 GPU 需求的计算框架**是一个先进工具，旨在评估和优化训练、微调以及部署大型语言模型（LLMs）所需的硬件需求。通过结合诸如内存估算、量化以及 GPU 基准测试等技术，该框架为开发者和研究人员提供了一种全面的方法论，以高效规划和分配计算资源。随着像 GPT-3、LLaMA-2 和 T5 这样的 LLM 不断在规模和复杂度上增长，精准估算 GPU 需求的能力对于确保其在研究和工业环境中的可访问性和可扩展性至关重要。

该框架提供了关键功能，包括适用于不同模型任务的内存计算器、硬件配置探索工具以及混合精度训练和模型并行化等高级策略。这些功能使用户能够根据模型大小、序列长度、批量大小和精度格式（例如 FP16 或 INT8）等因素预测 GPU 内存消耗。例如，对 LLaMA-2 13B 模型进行训练时，通过适当优化可能需要多块高容量 GPU，如 NVIDIA A100，而量化技术能够显著降低这些需求。框架还包含基准测试工具，可评估包括 NVIDIA 和 Apple Silicon 在内的多种架构的 GPU 性能，帮助用户选择最具性价比和效率的硬件解决方案。

除了资源估算，该框架还支持更广泛的应用，例如提升部署环境中的实时性能、进行硬件权衡分析，以及通过偏差检测和审计工具促进道德人工智能实践。框架设计友好，其中的 LLM 系统需求计算器（LLM System Requirements Calculator）简化了 GPU 需求的估算过程，无论是经验丰富的研究人员还是该领域的新手都能轻松操作。此外，通过整合如 GLUE 和 SuperGLUE 的综合基准，框架为从问答到翻译等任务提供了 LLM 性能的标准化评估。

该框架尤其值得注意的是，其针对 LLM 迅速扩展以及多样化硬件环境所面临的挑战提供了解决方案。通过在维持模型精度和吞吐量的同时优化 GPU 使用，该框架已成为推动开源大型语言模型开发和部署的重要资源。这使其在学术研究和工业应用中被广泛采用，确保了其在 LLM 持续推动自然语言处理边界的过程中保持相关性。

# 框架的关键特性

开源大型语言模型（LLMs）中用于估算GPU需求的计算框架包含多个关键功能，这些功能旨在简化和优化训练和部署任务的资源规划。

## 推理和训练的内存估算

该框架提供工具用于估算模型使用过程中各个阶段的GPU内存需求，包括推理、微调和完整训练。在推理阶段，主要的内存消耗来源于存储模型参数，而在训练过程中，还需要额外的资源用于优化器状态、梯度和激活内存[1][2]。例如，根据特定配置，运行一个LLaMA-2 13B模型至少需要三块A100 40GB GPU[3]。框架的LLM内存计算器允许用户输入模型参数并选择精度格式，例如FP32、FP16或INT8，从而能够估算这些任务的总内存使用量[4][2]。

## 优化的量化技术

量化是该框架的核心功能，它通过降低模型参数的精度来减少 GPU 内存需求，同时不会对模型精度产生显著影响。诸如 5 位量化等技术已被证明可以在处理大型模型（如 Airobors Llama-2）时显著降低内存使用，该模型在 300 亿（30B）参数配置下消耗了大约 23 GB 的 VRAM[5][3]。这一功能对于在各种 GPU 架构上高效部署模型至关重要。

## 支持多样化硬件配置

该框架评估并基准测试了广泛硬件上的GPU性能，包括Nvidia GPU和Apple Silicon M系列芯片。这些基准测试评估了在不同量化设置下，模型大小范围从70亿（7B）到750亿（75B）参数，帮助用户根据处理能力、内存带宽和容量做出明智的硬件决策[5][3]。理解这些因素之间的权衡对于确保GPU能够有效处理模型权重和计算而不出现瓶颈至关重要[6]。

## 高级GPU利用策略

为了解决大型语言模型（LLM）日益增长的复杂性，该框架提供了优化GPU利用率的策略。包括混合精度训练、激活检查点、梯度累积和模型并行等技术，以减少内存占用并提高计算吞吐量[7][8][9]。这些策略旨在防止内存不足错误、提升训练速度，并在保持模型精度的同时增强整体性能。

## 用户友好型工具和指标

该框架包括直观的工具，例如LLM系统需求计算器，这使得计算用于推理和训练任务的GPU内存需求的过程变得更加简单。它还支持人工评估和统计指标，例如BERTScore，提供了超越数值衡量的模型性能的细致见解[10][2]。这使开发者能够全面评估资源分配和模型效率。

## 全面基准测试

除了GPU内存估计之外，该框架还支持针对多种任务的基准测试，包括问答、编程和翻译。这些基准测试（如GLUE和SuperGLUE）提供了标准化的指标，如准确率和F1分数，用于评估LLM在各种场景中的能力[11][12]。加入伦理审计和偏见检测指标，确保该框架也关注负责任的人工智能实践[10]。  
通过整合内存估计、硬件基准测试、先进的优化技术和全面的评估指标，该框架为管理开源LLM的计算需求提供了一个全面的解决方案。  

# GPU 的架构基础



图形处理单元（GPUs）专为高效的并行计算而设计，使其成为深度学习任务（包括大型语言模型（LLMs）的训练和推理）的首选设备[6][13]。理解GPU架构对于优化性能和避免LLM部署过程中的瓶颈至关重要。GPU通过高内存带宽、专用核心（如张量核心）以及多线程并行性相结合，实现了高效性能，这使其能够大规模处理复杂计算[6][14]。

## GPU 架构的关键组件

### 内存带宽和容量

内存带宽和容量是GPU架构中用于LLM任务的最重要组成部分之一。这些指标会影响GPU高效处理模型权重和中间计算的能力。像NVIDIA A100这样的GPU，配备了高达80GB的高带宽内存（HBM2e），非常适合需要高内存容量的任务，例如运行大规模模型如GPT-3或LLaMA-2[15][16][3]。内存不足可能导致性能瓶颈，这突显了在部署LLM时使用具有充足VRAM的GPU的重要性[6][3]。

### 张量核心和并行处理

Tensor Cores 是 NVIDIA GPU（如 A100 和 H100）的一个特性，在加速矩阵运算方面发挥了关键作用，而矩阵运算是深度学习算法的基础[17][18]。这些专用核心支持高效的混合精度训练和推理，从而优化计算并减少内存使用[14]。此外，GPU 被设计为能够同时执行数千个线程，使其非常适合于自然语言处理、计算机视觉和基因组学等领域的并行处理任务[6][19]。

### 能源效率和可扩展性

能源效率是GPU架构的另一个重要方面，尤其是针对企业级LLM部署。现代GPU，例如AMD Instinct MI250X和Intel Data Center Max，在优化高内存任务的同时保持了能源效率，使其在成本敏感的环境中成为具有竞争力的选择[16]。对于大规模部署，像NVIDIA A100这样的GPU支持多实例GPU（MIG）功能，允许在多用户环境中实现共享工作负载和增强的可扩展性[16][20]。

## 架构权衡与优化

选择合适的GPU需要在处理能力、内存带宽和成本之间进行权衡。例如，虽然NVIDIA H100在推理任务中提供了无与伦比的性能，但对于某些用户来说，其成本可能过高[18]。另一方面，像NVIDIA A40这样的经济型选择为需求较低的任务提供了足够的性能[18]。为了最大化LLM部署的效率，用户还需要考虑架构优化，例如量化、混合精度训练，以及像TensorRT这样的高级推理库，这些都可以提升GPU资源的利用率[3][14]。

# GPU需求估算方法论

估算大型语言模型（LLMs）的GPU需求需要详细了解它们的内存需求、计算复杂性以及部署策略。该过程需要分析多个组件，例如模型大小、精度、批量大小、序列长度和硬件配置，以确保在推理或训练期间实现最佳性能。

## GPU内存估算公式

估算 GPU 需求的关键步骤是计算推理和训练所需的内存需求。
\[
\text{总内存} = \text{模型大小} + \text{KV 缓存} + \text{激活值} + (\text{优化器状态} + \text{梯度}) \times \text{可训练参数的数量}
\]
该公式考虑了存储模型权重、键值 (KV) 缓存和中间激活值所需的内存。在训练过程中，还需要额外的内存来存储优化器状态和计算梯度。例如，训练像 LLaMA-13B 这样的模型不仅需要存储参数的内存，还需要为 KV 缓存和其他开销分配额外的资源[1][2]。

## 影响GPU需求的因素

### 模型参数和精度

模型参数的大小是GPU内存消耗的主要决定因素。例如，GPT-3的1750亿参数相比于早期的模型（如只有1.1亿参数的BERT）需要显著更多的内存[21]。内存使用量还可能因模型的精度（例如，FP16 与 FP32）而有所不同，较低精度的格式（如 int8 或 int4）可以在推理过程中帮助减少内存需求[1][22]。

### 序列长度和批量大小

序列长度和批量大小在确定GPU需求方面起着关键作用。较长的序列或更大的批量需要额外的内存来存储激活值和KV缓存。对这些组件进行高效的内存分配可以显著提高性能，从而使系统能够处理更大的批量而不超出GPU容量[23][24]。

### 硬件考虑与模型并行性

如果估算的内存需求超过了单个GPU的容量，则必须采用模型并行或分片等策略。这些技术将模型分布到多个GPU上，从而能够在单个GPU容量有限的硬件上部署像LLaMA-13B或GPT-4这样的大规模模型[23][25]。

## 实际部署注意事项

在实际应用中，现代框架提供了优化 GPU 使用和减少内存瓶颈的工具。量化、精度调整以及为 KV 缓存动态分配内存等技术经常被应用。例如，NeuSight 框架可以预测不同 LLM 在未知 GPU 上的性能，从而为特定工作负载提供最佳配置的洞察[26][27]。

此外，像 LLM 系统需求计算器这样的内存估算工具，通过提供用户友好的界面简化了评估推理和训练任务所需内存的过程。这些工具允许用户调整精度、批大小和序列长度等参数，促进更高效的资源分配和部署[2]。

## 性能评估和成本效率

为了确保成本与性能的平衡，在推理过程中使用诸如每百万个令牌的价格和每秒输出令牌率（TPS）等指标来评估GPU利用率。通过监控波动的GPU利用率——即GPU工作负载的一个衡量标准——用户可以优化配置以实现最大效率。

# 框架的应用

用于估算开源大型语言模型（LLMs）GPU需求的计算框架在研究、开发和部署环境中具有多种应用。通过考虑批量大小、模型结构、内存要求和计算复杂性等参数，该框架可以为LLMs的训练、微调和推理提供精确的资源规划[28][7]。

## 高效的模型训练和微调

该框架在模型训练和微调过程中优化GPU利用率方面起着重要作用。例如，它可以通过考虑批量大小、训练时长和资源可用性，计算像Mixtral Instruct 7B这样的模型所需的确切GPU内存需求。这使开发者能够估算出大规模训练所需的GPU数量，并避免内存不足错误[28][7][29]。诸如混合精度训练、梯度累积以及激活检查点等技术可以集成到框架中，以进一步提高GPU效率并在训练过程中减少内存开销[7][8][29]。

## 模型部署的硬件选择

选择合适的硬件是部署LLMs时一个关键的考虑因素，而该框架提供了一种系统化的方法来评估GPU选项。通过分析内存带宽和张量核心利用率等关键性能指标，开发者可以选择最符合其计算需求的GPU。例如，像Hugging Face的T5这样的较小模型可能在消费级GPU上高效运行，而较大的模型如GPT-3或wav2vec则需要高内存的GPU（如NVIDIA A100）来应对其巨大的处理需求[15][6]。该框架简化了这一选择过程，确保在部署期间实现硬件的最佳利用。

## 实时应用优化

在实时应用中，该框架有助于管理响应延迟和上下文大小需求。它评估支持多个并发请求所需的总令牌处理能力，从而帮助组织优化其 GPU 配置以实现低延迟和高吞吐量场景。例如，它可以根据 `kv_cache_size_per_token` 和平均上下文窗口大小计算内存占用，确保在面对大规模用户需求时实现无缝的实时处理[30][25]。

## 行业与开源进展

该框架通过在开源环境中比较不同的GPU，支持更广泛的行业和学术计划。这种方式通过使开发者能够在各种硬件配置上优化模型并识别架构改进的领域，促进了创新和协作[31]。此外，它鼓励开发开源的GPU编程框架，从而提高研究人员和开发人员使用的透明性和可访问性[31]。  
通过整合这些多样化的应用，该计算框架提高了使用开源大型语言模型的效率、可扩展性和实用性，从而解决了其部署和使用中的关键挑战。  